<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Final-Project-Check-in" data-toc-modified-id="Final-Project-Check-in-1">Final Project Check-in</a></span></li><li><span><a href="#Group-Name" data-toc-modified-id="Group-Name-2">Group Name</a></span></li><li><span><a href="#Student-Names" data-toc-modified-id="Student-Names-3">Student Names</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-4">Load Data</a></span></li><li><span><a href="#Fit-scikit-learn-model" data-toc-modified-id="Fit-scikit-learn-model-5">Fit scikit-learn model</a></span></li><li><span><a href="#Evaluation-Metric" data-toc-modified-id="Evaluation-Metric-6">Evaluation Metric</a></span></li></ul></div>

Final Project Check-in
------

Group Name
-----

Litter Box

Student Names
----

1. Ruizhe (Jack) Dong
2. Xu Liu
3. Esme Luo

Load Data
-----

In [17]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import median_absolute_error
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv("Data/train.csv")

y = data.SalePrice
X = data.drop(['SalePrice'], axis=1)


Fit scikit-learn model
----

In [18]:
def make_pipelines(regressor=None):
    "Create a single pipeline that processing the data and then fits the regressor." 
        
    numeric_features = ['1stFlrSF','2ndFlrSF','3SsnPorch','BedroomAbvGr','BsmtFinSF1','BsmtFullBath','BsmtHalfBath','BsmtUnfSF',
                       'EnclosedPorch','Fireplaces','FullBath','GarageArea','GarageCars','GarageYrBlt','GrLivArea','HalfBath',
                       'KitchenAbvGr','LotArea','LotFrontage','LowQualFinSF','MasVnrArea','MiscVal','MoSold','MSSubClass','OpenPorchSF',
                       'OverallCond','OverallQual','PoolArea','ScreenPorch','YrSold', 'YearRemodAdd', 'YearBuilt', 'WoodDeckSF', 
                        'TotRmsAbvGrd', 'TotalBsmtSF']
    numeric_transformer = Pipeline(steps=[('imputer', impute.SimpleImputer(strategy='median'))])
    
    categorical_features = ['BldgType','BsmtCond','BsmtExposure','BsmtFinType1','BsmtQual','CentralAir','Condition1','Condition2',
                           'Electrical','ExterCond','Exterior1st','Exterior2nd','ExterQual','Foundation','Functional','GarageType',
                           'Heating','HeatingQC','HouseStyle','KitchenQual','LandContour','LandSlope','LotConfig','LotShape',
                           'MasVnrType','MiscFeature','MSZoning','Neighborhood','PavedDrive','RoofMatl','RoofStyle','SaleCondition',
                           'SaleType','Street','Utilities']
    categorical_transformer = Pipeline(steps=[('imputer', impute.SimpleImputer(strategy='constant', fill_value='most_frequent')),
                                              ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])
    
    
    preprocessor = compose.ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features),
                                                           ('cat', categorical_transformer, categorical_features)])
    
    pipe_rf = Pipeline([('preprocessor', preprocessor),
                        ('clf', RandomForestRegressor(n_estimators=30, min_samples_leaf=1, random_state=42))])
    pipe_lasso = Pipeline([('preprocessor', preprocessor),
                           ('clf', linear_model.Lasso(alpha=0.1))])
    pipe_gbr = Pipeline([('preprocessor', preprocessor),
                           ('clf', GradientBoostingRegressor())])
    
    pipeline = [pipe_rf, pipe_lasso, pipe_gbr]

    
    return pipeline

In [19]:
pipelines = make_pipelines()

Evaluation Metric
----

In [21]:
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
#from sklearn.metrics import m

In [22]:
def MAPE(predict,target):
    return (abs((target - predict) / target).mean()) * 100

In [23]:
def mae(pipelines, X, y):
    d = dict()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    for pipe in pipelines:
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        name = pipe.steps[-1][1].__class__.__name__.split('.')[-1]
        mae = round(median_absolute_error(y_test, y_pred), 2)
        mse = round(mean_squared_error(y_test, y_pred), 2)
        r2 = round(r2_score(y_test, y_pred), 2)
        mape = MAPE(y_test, y_pred)
        d[name] = [mae, mse, r2, mape]
    return d

In [24]:
d = mae(pipelines, X, y)

In [25]:
df = pd.DataFrame(d.values(), d.keys())

In [26]:
df.columns = ['MAE', 'MSE', 'R^2', 'MAPE']

In [27]:
df

,MAE,MSE,R^2,MAPE
RandomForestRegressor,10397.42,9.653509e+08,0.87,9.508605
Lasso,12192.68,8.379658e+08,0.89,11.693062
GradientBoostingRegressor,11327.05,9.394366e+08,0.87,9.247985
